<a href="https://colab.research.google.com/github/vigneshjayanth00/Football/blob/master/FIFA_new_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from IPython.display import Markdown
from scipy import stats

'''Seaborn and Matplotlib Visualization'''
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('bmh')                    
%matplotlib inline
from DataFrameImputer import DataFrameImputer
'''plotly Visualization'''
import plotly.offline as py
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected = True)
from sklearn.base import TransformerMixin
%time

from sklearn.impute import KNNImputer as knn_impute
import pandas as pd
import glob


Wall time: 0 ns


In [0]:
import os

path = r'C:\Users\Home\Downloads\All Fifa Datasets\Datasets' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df['Year'] = os.path.basename(filename)
    li.append(df)

df_combined = pd.concat(li, axis=0, ignore_index=True)

display(print('Dimension of data:', df_combined.shape))

Dimension of data: (100995, 105)


None

In [0]:
df_combined.Year=df_combined.Year.str[:4]
df_combined.Year.unique()

array(['2015', '2016', '2017', '2018', '2019', '2020'], dtype=object)

In [0]:
Goalkeeper_cols=list(df_combined.filter(regex='gk').columns)
Goalkeeper_cols


['gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_reflexes',
 'gk_speed',
 'gk_positioning']

In [0]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100995 entries, 0 to 100994
Columns: 105 entries, sofifa_id to Year
dtypes: float64(16), int64(11), object(78)
memory usage: 80.9+ MB


In [0]:
#Impute missing values
df_combined= DataFrameImputer().fit_transform(df_combined)
df_combined.team_position.unique()

array(['CF', 'LW', 'SUB', 'ST', 'GK', 'RES', 'LM', 'RS', 'RW', 'LS',
       'RCM', 'LCB', 'CAM', 'LCM', 'LDM', 'RCB', 'CM', 'CDM', 'CB', 'RDM',
       'RB', 'RM', 'LB', 'RAM', 'LAM', 'RWB', 'RF', 'LF', 'LWB'],
      dtype=object)

In [0]:
#Checking which columns are of different dtype

float_df=df_combined.loc[:, df_combined.dtypes == np.float64]
int_df=df_combined.loc[:, df_combined.dtypes == np.int64]
string_df=df_combined.loc[:, df_combined.dtypes == np.object]
int_df.columns

Index(['sofifa_id', 'age', 'height_cm', 'weight_kg', 'overall', 'potential',
       'value_eur', 'wage_eur', 'international_reputation', 'weak_foot',
       'skill_moves'],
      dtype='object')

In [0]:
#Iterating through the dataframe to check unique values across columns with different dtypes
A=[]
B=[]
C=[]
D=[]
E=[]
F=[]

for col in float_df:
    A.append(col)
    B.append(float_df[col].unique())
    
for col in string_df:
    C.append(col)
    D.append(string_df[col].unique())

for col in int_df:
    E.append(col)
    F.append(int_df[col].unique())
    
Unique_float_df_summary = pd.DataFrame(
    {'Column_Name': A,
     'Unique_Values': B
    })
Unique_string_df_summary = pd.DataFrame(
    {'Column_Name': C,
     'Unique_Values': D
    })
Unique_int_df_summary = pd.DataFrame(
    {'Column_Name': E,
     'Unique_Values': F
    })
Unique_float_df_summary.to_csv(r'C:\Users\Home\Downloads\All Fifa Datasets\Data Cleaning\Unique_float_df_summary.csv')
Unique_string_df_summary.to_csv(r'C:\Users\Home\Downloads\All Fifa Datasets\Data Cleaning\Unique_string_df_summary.csv')
Unique_int_df_summary.to_csv(r'C:\Users\Home\Downloads\All Fifa Datasets\Data Cleaning\Unique_int_df_summary.csv')

Unique_string_df_summary

,Column_Name,Unique_Values
0,player_url,[https://sofifa.com/player/158023/lionel-messi...
1,short_name,"[L. Messi, Cristiano Ronaldo, A. Robben, Z. Ib..."
2,long_name,"[Lionel Andrés Messi Cuccittini, Cristiano Ron..."
3,dob,"[1987-06-24, 1985-02-05, 1984-01-23, 1981-10-0..."
4,nationality,"[Argentina, Portugal, Netherlands, Sweden, Ger..."
...,...,...
73,lcb,"[45+3, 52+3, 46+3, 55+3, 63+2, 57+3, 46+2, 48+..."
74,cb,"[45+3, 52+3, 46+3, 55+3, 63+2, 57+3, 46+2, 48+..."
75,rcb,"[45+3, 52+3, 46+3, 55+3, 63+2, 57+3, 46+2, 48+..."
76,rb,"[54+3, 57+3, 55+3, 56+3, 61+2, 62+3, 54+2, 81+..."


In [0]:
df_combined1=df_combined
df_combined1=df_combined1.sort_values(['value_eur'], ascending=[False])

In [0]:
#These columns need to be removed as they are either related to GK stats or irrelevant information for modelling
Remove_Columns=['sofifa_id','player_url','long_name','dob','real_face','player_tags','team_jersey_number',
'loaned_from','joined','nation_position','nation_jersey_number','player_traits','goalkeeping_diving','goalkeeping_handling','goalkeeping_kicking',
'goalkeeping_positioning','goalkeeping_reflexes','gk_diving', 'gk_handling','gk_kicking', 'gk_reflexes', 'gk_speed', 'gk_positioning',
                'contract_valid_until','release_clause_eur','player_positions']

df_combined1.drop(Remove_Columns, axis=1, inplace=True)

In [0]:
#These columns have numbers stored as strings with an added number such as '93+2'
String_Columns1=['ls','st','rs','lw','lf','cf','rf','rw','lam','cam','ram','lm','lcm','cm','rcm','rm',
                'lwb','ldm','cdm','rdm','rwb','lb','lcb','cb','rcb','rb']

for col in String_Columns1:
    df_combined1[col] = df_combined1[col].str[:2]
    df_combined1[col] = df_combined1[col].astype(int)


In [0]:
#These columns have hypenated numbers stored as strings such as '6-'
String_Columns=['attacking_crossing','attacking_finishing','attacking_heading_accuracy','attacking_short_passing',
'attacking_volleys','skill_dribbling','skill_curve','skill_fk_accuracy','skill_long_passing',
'skill_ball_control','movement_acceleration','movement_sprint_speed','movement_agility',
'movement_reactions','movement_balance','power_shot_power','power_jumping','power_stamina',
'power_strength','power_long_shots','mentality_aggression','mentality_interceptions','mentality_positioning',
'mentality_vision','mentality_penalties','mentality_composure','defending_marking','defending_standing_tackle',
'defending_sliding_tackle']

for col in String_Columns:
    df_combined1[col] = df_combined1[col].replace('\-', '', regex=True)
    df_combined1[col] = df_combined1[col].replace('\+', '', regex=True)
    df_combined1[col] = df_combined1[col].str[:2]
    df_combined1[col] = df_combined1[col].fillna(0).astype(int).replace(to_replace=0, method='ffill')

In [0]:
df_combined1=df_combined1.reset_index(drop=True)


In [0]:
df_combined2=df_combined1[df_combined1['team_position']!='GK']

df_combined2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97339 entries, 0 to 100994
Data columns (total 79 columns):
short_name                    97339 non-null object
age                           97339 non-null int64
height_cm                     97339 non-null int64
weight_kg                     97339 non-null int64
nationality                   97339 non-null object
club                          97339 non-null object
overall                       97339 non-null int64
potential                     97339 non-null int64
value_eur                     97339 non-null int64
wage_eur                      97339 non-null int64
preferred_foot                97339 non-null object
international_reputation      97339 non-null int64
weak_foot                     97339 non-null int64
skill_moves                   97339 non-null int64
work_rate                     97339 non-null object
body_type                     97339 non-null object
team_position                 97339 non-null object
pace            

In [0]:
df_combined2.overall.unique()

array([92, 94, 93, 91, 89, 90, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 77,
       78, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61,
       60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 49, 48, 50, 47, 46, 45, 44,
       40, 41, 42, 43], dtype=int64)

In [0]:
A=[]
B=[]

for col in df_combined2:
    A.append(col)
    B.append(df_combined2[col].unique())
    
Unique_values_summary = pd.DataFrame(
    {'Column_Name': A,
     'Unique_Values': B
    })

In [0]:
Unique_values_summary.to_csv(r'C:\Users\Home\Downloads\All Fifa Datasets\Data Cleaning\Unique_values_summary.csv')

In [0]:
float_cols=['pace','shooting','passing','dribbling','defending','physic']

for col in float_cols:
    df_combined2[col] = round(df_combined2[col]).astype(int)
    
    

C:\Users\Home\Anaconda3\envs\venv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [0]:
clubs=pd.DataFrame(df_combined2.club.unique())
clubs.to_csv(r'C:\Users\Home\Downloads\All Fifa Datasets\Data Cleaning\Clubs_list.csv')

In [0]:
df_combined2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97339 entries, 0 to 100994
Data columns (total 79 columns):
short_name                    97339 non-null object
age                           97339 non-null int64
height_cm                     97339 non-null int64
weight_kg                     97339 non-null int64
nationality                   97339 non-null object
club                          97339 non-null object
overall                       97339 non-null int64
potential                     97339 non-null int64
value_eur                     97339 non-null int64
wage_eur                      97339 non-null int64
preferred_foot                97339 non-null object
international_reputation      97339 non-null int64
weak_foot                     97339 non-null int64
skill_moves                   97339 non-null int64
work_rate                     97339 non-null object
body_type                     97339 non-null object
team_position                 97339 non-null object
pace            

In [0]:
df_combined2.to_csv(r'C:\Users\Home\Downloads\All Fifa Datasets\Updated Dataset after Cleaning\Updated1.csv')

In [0]:
import plotly.express as px

df = px.data.wind()
fig = px.bar_polar(df, r="frequency", theta="direction",
                   color="strength",
                   color_discrete_sequence= px.colors.sequential.Sunset)
fig.show()